# 1 TextRank实体抽取类设计与实现
---------------------------------------------------------------------
### 1.1 基于窗口共现的TextRank类实现

Equation: $WS(V_i)=(1 - d)\frac{1}{N} + d \sum_{V_j \in In(V_i)} \frac {W_{ji}}{\sum_{V_k \in Out(V_j)}} WS(V_j)$

In [1]:
# 基于窗口共现的TextRank类
class WinBasedTextRank:
    def __init__(self, double_word_list):
        self.double_word_list = double_word_list
        # 构建词表
        self.all_words = self.__get_all_words()
        # 键值对：词->窗口内词集
        self.win_dict = self.__get_win_dict()
        self.score_dict = self.__get_TextRank_score_dict()
        
    def __get_all_words(self):
        words = []
        for sent in self.double_word_list:
            for word in sent:
                words.append(word)
        
        return words
    
    def __get_win_dict(self):
        win_dict = {}
        for i in range(len(self.all_words)):
            if self.all_words[i] not in win_dict.keys():
                win_dict[self.all_words[i]] = set() 
            lindex = i - 5
            if lindex < 0:
                lindex = 0
            rindex = i + 5 
            if rindex > len(self.all_words) - 1:
                rindex = len(self.all_words) - 1
            for wd in self.all_words[lindex : rindex]:
                win_dict[self.all_words[i]].add(wd)
        
        return win_dict
    
    def __get_TextRank_score_dict(self):
        time = 0
        score_dict = {w:0.99/len(self.all_words) for w in self.all_words}
        while(time < 50):
            for k, v in self.win_dict.items():
                s = score_dict[k] / len(v)
                score_dict[k] = 0
                for i in v:
                    score_dict[i] += 0.01*s
            time += 1
        
        return score_dict    
    
    def get_TextRank_kv(self, text):
        temp_dict = {}
        for word in text:
            if word in self.score_dict.keys():
                temp_dict[word] = (self.score_dict[word])
        values_list = sorted(temp_dict.items(), key=lambda temp_dict:temp_dict[1], reverse=True) #TextRank从大到小排序
    
        return values_list
    
    def get_TextRank_k(self, text):
        values_list = self.get_TextRank_kv(text)
        key_list = []
        for key in values_list:
            key_list.append(value[0])
        
        return (key_list)

### 1.2 基于FastText词向量计算相似度的TextRank实现
### pagerank equation: $r_j = \sum_{i \rightarrow j} \beta \frac {r_i} {d_i} + (1 - \beta) \frac {1}{N}$
##### FastText Model类定义

In [2]:
from gensim.models.fasttext import FastText
import time

class FTModel:
    def __init__(self, model_path):
        print('FastText word2vec model loading...', end='')
        start = time.time()
        self.model = FastText.load_fasttext_format(model_path)
        end = time.time()
        print('successfully, elapsed: %.2f s' % (end - start))

##### 加载FastText词向量模型

In [3]:
model_path = '../../../models/FastText/cc.zh.300.bin.gz'
ft_model = FTModel(model_path)

FastText word2vec model loading...

/tmp/ipykernel_21196/3929849022.py:8: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  self.model = FastText.load_fasttext_format(model_path)


successfully, elapsed: 483.06 s


##### FastText相似的TextRank类代码实现

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from keras.preprocessing.text import Tokenizer

class FTSimBasedTextRank:
    def __init__(self, model, double_word_list):
        self.ft_model = model
        self.double_word_list = double_word_list
        # 构建词表
        self.all_words = self.__get_all_words()
        self.word_embeddings = self.__get_embeddings()
        self.word_sim_mat = self.__get_word_sim_mat()
 
    def __get_all_words(self):
        words = []
        for sent in self.double_word_list:
            for word in sent:
                words.append(word)
        
        return words
    
    def __get_embeddings(self):
        word_embeddings = {}
        for word in self.all_words:
            try:
                # model.wv[word]存的就是这个word的词向量
                word_embeddings[word] = self.ft_model.model.wv[word]
            except KeyError:
                continue
        return word_embeddings
    
    def __get_word_sim_mat(self):
        word_sim_vectors = []
        for _, v in self.word_embeddings.items():
            word_sim_vectors.append(v)
        word_sim_mat = np.zeros([len(word_sim_vectors), len(word_sim_vectors)])
        for i in range(len(word_sim_vectors)):
            for j in range(len(word_sim_vectors)):
                if i != j:
                    word_sim_mat[i][j] = cosine_similarity(word_sim_vectors[i].reshape(1, 300), word_sim_vectors[j].reshape(1, 300))[0, 0]
        return word_sim_mat
    
    def get_vocab(self):
        #fit_on_texts函数可以将输入的文本中的每个词编号，
        #编号是根据词频的，词频越大，编号越小
        tokenizer=Tokenizer()
        tokenizer.fit_on_texts(self.double_word_list)
        vocab = tokenizer.word_index  # 得到每个词的编号
        return vocab

    def get_TextRank_kv(self):
        word_nx_graph = nx.from_numpy_array(self.word_sim_mat)
        word_scores = nx.pagerank(word_nx_graph)
        ranked_words = sorted(((word_scores[i], s) for i,s in enumerate(self.get_vocab())), reverse=True)
        return ranked_words
    
    def get_TextRank_k(self):
        ranked_words = get_TextRank_kv()
        keys = []
        for v, k in ranked_words:
            keys.append(k)
        return keys
    

2022-01-18 02:45:51.944160: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 02:45:51.945306: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 2 数据处理

### 2.1 爬取网上一篇文章并保存

In [46]:
import requests
from lxml import html

etree=html.etree
#url='http://news.cri.cn/gb/27824/2013/01/28/6611s4005015.htm'
url='https://news.sina.com.cn/gov/xlxw/2022-01-18/doc-ikyakumy1103837.shtml'
data=requests.get(url)
data.encoding='utf-8'
#print(data)
s=etree.HTML(data.text)
# text1=s.xpath('//div[@id="ccontent"]/p/text()')#得到的文本是一个列表，里面有6项，代表6个自然段
# title=s.xpath('//div[@class="news_ts_tit"]/text()')[0].strip()#[0]是取标题的第一项，trip()去掉首尾空格
text1=s.xpath('//div[@id="article"]/p/text()')#得到的文本是一个列表，里面有6项，代表6个自然段
title=s.xpath('//h1[@class="main-title"]/text()')[0].strip()#[0]是取标题的第一项，trip()去掉首尾空格
print("爬取文本：\n","标题：",title,"\n正文：",text1)
text=title

# 将得到的文本写入文件
for i in range(0,len(text1)-1):
   text += text1[i]
sentence_list=[]
print(text)
title='dat/' + title + '.txt'
with open(title, 'w', encoding='utf-8') as f:
   f.writelines(text)

爬取文本：
 标题： 回答世界之问，习近平四个比喻发人深省 
正文： ['\u3000\u3000原标题：第一观察 | 回答世界之问，习近平四个比喻发人深省', '\u3000\u3000五年前，1月17日，习近平主席在瑞士达沃斯出席世界经济论坛2017年年会，面对“世界怎么了、我们怎么办”的世界之问，纵论经济全球化走向，为迷茫困顿中的人们带来信心和力量。', '\u3000\u3000这一次，也是1月17日，习近平主席出席2022年世界经济论坛视频会议并发表演讲，面对“如何战胜疫情？如何建设疫后世界？”的又一世界之问，呼吁各方坚定信心、勇毅前行，为共创后疫情时代美好世界指明方向。', '\u3000\u3000从习近平主席演讲中提到的“大船”之喻、“大江”之喻、“蛋糕”之喻、“虎年”之喻中，我们感受着在“深刻而宏阔的时代之变”面前，中国领导人的历史和哲学思索，远见、自信与担当。', '\u3000\u3000', '\u3000\u3000“我们同在一条船上”——2018年11月，出席在太平洋探索者号邮轮上举行的亚太经合组织工商领导人峰会，习近平主席曾以“船”为喻，呼吁各方“同舟共济”，共同应对风险挑战。', '\u3000\u3000今天，习近平主席再次用到“船”的比喻。这次，他谈到的是“小船”和“大船”的关系。', '\u3000\u3000经历了持续延宕的新冠肺炎疫情，人类比以往更加感受到何为“同舟共济”。在一个“黑天鹅”和“灰犀牛”频频造访的世界，在各种可以预见的狂风暴雨和难以想象的惊涛骇浪面前，“小船”只会风雨飘摇，“巨舰”才能破浪前行。', '\u3000\u3000然而，并不是所有人都有这样的历史自觉。在疫情等全球性挑战面前，相互掣肘、无端“甩锅”者有之，贻误战机、干扰大局者有之，煽动仇恨偏见、进行围堵打压甚至对抗者有之……凡此种种，都无助于战胜共同面临的危机。', '\u3000\u3000“大船”之喻，正是人类命运共同体之喻。', '\u3000\u3000众人划桨开大船。', '\u3000\u3000于潮平海阔之时拉紧彼此，是瞭望者的历史远见；', '\u3000\u3000于风狂雨骤之时把握方向，是领航者的历史担当。', '\u3000\u3000', '\u3000\u3000“青山遮不住，毕竟东流去。”', '\u3000\u3000中

### 2.2 打开文件

In [48]:
import numpy as np
import re,jieba.posseg as pseg
from itertools import chain

#打开文件
sentences_list = []
file_path=title
fp = open(file_path,'r',encoding="utf8")
for line in fp.readlines():
        if line.strip():
            # 把元素按照[。！；？]进行分隔，得到句子。
            line_split = re.split(r'[。！；？]',line.strip())
            # [。！；？]这些符号也会划分出来，把它们去掉。
            line_split = [line.strip() for line in line_split if line.strip() not in ['。','！','？','；'] and len(line.strip())>1]
            sentences_list.append(line_split)
sentences_list = list(chain.from_iterable(sentences_list))
print("前10个句子为：\n")
print(sentences_list[:10])
print("句子总数：", len(sentences_list))

前10个句子为：

['回答世界之问，习近平四个比喻发人深省\u3000\u3000原标题：第一观察 | 回答世界之问，习近平四个比喻发人深省\u3000\u3000五年前，1月17日，习近平主席在瑞士达沃斯出席世界经济论坛2017年年会，面对“世界怎么了、我们怎么办”的世界之问，纵论经济全球化走向，为迷茫困顿中的人们带来信心和力量', '这一次，也是1月17日，习近平主席出席2022年世界经济论坛视频会议并发表演讲，面对“如何战胜疫情', '如何建设疫后世界', '”的又一世界之问，呼吁各方坚定信心、勇毅前行，为共创后疫情时代美好世界指明方向', '从习近平主席演讲中提到的“大船”之喻、“大江”之喻、“蛋糕”之喻、“虎年”之喻中，我们感受着在“深刻而宏阔的时代之变”面前，中国领导人的历史和哲学思索，远见、自信与担当', '“我们同在一条船上”——2018年11月，出席在太平洋探索者号邮轮上举行的亚太经合组织工商领导人峰会，习近平主席曾以“船”为喻，呼吁各方“同舟共济”，共同应对风险挑战', '今天，习近平主席再次用到“船”的比喻', '这次，他谈到的是“小船”和“大船”的关系', '经历了持续延宕的新冠肺炎疫情，人类比以往更加感受到何为“同舟共济”', '在一个“黑天鹅”和“灰犀牛”频频造访的世界，在各种可以预见的狂风暴雨和难以想象的惊涛骇浪面前，“小船”只会风雨飘摇，“巨舰”才能破浪前行']
句子总数： 57


### 2.3 分词、停用词

In [49]:
# 加载停用词
stoplist= [word.strip() for word in open('stopwords.txt',encoding='utf-8').readlines()]
# print(stoplist)
word_flag = {}

# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = pseg.cut(sentence.strip())
    word_list = []
    for word,flag in sentence_depart:
        if word not in stoplist:
            word_list.append(word)
            word_flag[word] = flag
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

sentence_word_list = []
for sentence in sentences_list:
    line_seg = seg_depart(sentence)
    sentence_word_list.append(line_seg)
print("一共有",len(sentences_list),'个句子。\n')
print("前10个句子分词后的结果为：\n",sentence_word_list[:10])

# 保证处理后句子的数量不变，我们后面才好根据textrank值取出未处理之前的句子作为摘要。
if len(sentences_list) == len(sentence_word_list):
    print("\n数据预处理后句子的数量不变！")

一共有 57 个句子。

前10个句子分词后的结果为：
 [['回答', '世界', '问', '习近平', '四个', '比喻', '发人深省', '原', '标题', '第一', '观察', '回答', '世界', '问', '习近平', '四个', '比喻', '发人深省', '五年', '前月日', '习近平', '主席', '瑞士', '达沃斯', '出席', '世界', '经济', '论坛', '年', '年', '会', '面对', '世界', '世界', '问', '纵论', '经济', '全球化', '走向', '迷茫', '困顿', '中', '人们', '带来', '信心', '力量'], ['一次', '月', '日', '习近平', '主席', '出席', '年', '世界', '经济', '论坛', '视频会议', '发表', '演讲', '面对', '战胜', '疫情'], ['建设', '疫后', '世界'], ['世界', '问', '呼吁', '各方', '坚定信心', '勇毅', '前', '行为', '共创', '后', '疫情', '时代', '美好世界', '指明方向'], ['习近平', '主席', '演讲', '中', '提到', '大船', '喻', '大江', '喻', '蛋糕', '喻', '虎年', '喻', '中', '感受', '深刻', '宏阔', '时代', '变', '面前', '中国', '领导人', '历史', '哲学', '思索', '远见', '自信', '担当'], ['一条', '船上', '年', '月', '出席', '太平洋', '探索者', '号', '邮轮', '上', '举行', '亚太经合组织', '工商', '领导人', '峰会', '习近平', '主席', '曾', '船', '喻', '呼吁', '各方', '同舟共济', '共同', '应对', '风险', '挑战'], ['今天', '习近平', '主席', '再次', '用到', '船', '比喻'], ['这次', '谈到', '小船', '大船', '关系'], ['经历', '持续', '延宕', '新冠', '肺炎', '疫情', '人类', '以往', '更加', '感受', '何为', '同舟共济'],

# 3 实体抽取
### 3.1 窗口共现

In [50]:
word_list = []
for sent in sentence_word_list:
    for word in sent:
        word_list.append(word)
        
win_tr = WinBasedTextRank(sentence_word_list)
win_kvs = win_tr.get_TextRank_kv(word_list)

i = 0
cnt = 0
for k, v in win_kvs[:int(len(win_kvs)*.95)]:
    fg = word_flag[k]
    if (fg.startswith('n') and len(fg)>1) or fg.startswith('t') or fg.startswith('s'):
        cnt += 1
        print(cnt, "\t", i+1, "\t", (k,v), '\t', fg)
    i += 1

1 	 2 	 ('习近平', 3.3827774991418443e-121) 	 nrfg
2 	 13 	 ('喻', 1.1171176442893095e-121) 	 nr
3 	 24 	 ('中国', 7.611595277588015e-122) 	 ns
4 	 27 	 ('达沃斯', 6.481323485700361e-122) 	 nr
5 	 28 	 ('五年', 6.453631707698266e-122) 	 t
6 	 44 	 ('前月日', 4.957337940159634e-122) 	 t
7 	 46 	 ('黑天鹅', 4.5767379093499387e-122) 	 nz
8 	 47 	 ('瑞士', 4.5331473754738465e-122) 	 ns
9 	 49 	 ('太平洋', 4.3306608142406035e-122) 	 ns
10 	 51 	 ('今天', 4.1628399353490984e-122) 	 t
11 	 62 	 ('探索者', 3.6517119249845045e-122) 	 nr
12 	 65 	 ('以往', 3.568954650545539e-122) 	 t
13 	 72 	 ('虎年', 3.212775901700971e-122) 	 t
14 	 76 	 ('大江', 3.124390260632934e-122) 	 ns
15 	 86 	 ('船上', 2.662247718952779e-122) 	 s
16 	 96 	 ('大海', 2.2731492066608617e-122) 	 ns
17 	 112 	 ('亚太经合组织', 1.5970845099530778e-122) 	 nt
18 	 128 	 ('时', 1.0022870908674213e-122) 	 ng
19 	 133 	 ('湖泊', 8.747212210813152e-123) 	 ns
20 	 150 	 ('青山', 6.452771904432765e-123) 	 ns
21 	 156 	 ('广度', 5.924341191058235e-123) 	 ns
22 	 157 	 ('潮平海', 5.8661

### 3.2 FastText词向量相似度

In [51]:
ftsim_tr = FTSimBasedTextRank(ft_model, sentence_word_list)
ftsim_kvs = ftsim_tr.get_TextRank_kv()

i = 0
cnt = 0
for v, k in ftsim_kvs[:int(len(ftsim_kvs)*.95)]:
    fg = word_flag[k]
    if (fg.startswith('n') and len(fg)>1) or fg.startswith('t') or fg.startswith('s'):
        cnt += 1
        print(cnt, "\t", i+1, "\t", (k,v), '\t', fg)
    i += 1

1 	 24 	 ('宝贵', 0.002742921409241907) 	 nr
2 	 29 	 ('五年', 0.0027293775653264925) 	 t
3 	 51 	 ('新春', 0.0026291983065427817) 	 ns
4 	 74 	 ('改革开放', 0.0025576802178185433) 	 nz
5 	 99 	 ('深度', 0.002475588789583414) 	 ns
6 	 112 	 ('努力实现', 0.002442140037452893) 	 nr
7 	 122 	 ('新华社', 0.0024129291292472425) 	 nt
8 	 143 	 ('以往', 0.00236739061339443) 	 t
9 	 150 	 ('杨依军', 0.0023492412172824084) 	 nr
10 	 159 	 ('船上', 0.0023332539441397403) 	 s
11 	 164 	 ('筑墙', 0.0023223507902135126) 	 nz
12 	 166 	 ('大海', 0.0023172749795044145) 	 ns
13 	 173 	 ('时', 0.002306819914010548) 	 ng
14 	 189 	 ('中虎', 0.0022818342755228176) 	 nz
15 	 196 	 ('日内瓦', 0.0022693665588778794) 	 ns
16 	 197 	 ('中国共产党', 0.0022669348725534925) 	 nt
17 	 204 	 ('赵承', 0.002256275526432507) 	 nr
18 	 207 	 ('智慧', 0.002255556223351494) 	 nr
19 	 208 	 ('碳达峰', 0.0022542864614083953) 	 nz
20 	 212 	 ('巨变', 0.002250422305440314) 	 nz
21 	 220 	 ('大江', 0.002236526919691958) 	 ns
22 	 222 	 ('黑天鹅', 0.0022346143364660597) 	 nz
23 	

### 3.3 集成实体抽取结果

In [52]:
win_entities = []
for k, v in win_kvs[:int(len(win_kvs)*.95)]:
    fg = word_flag[k]
    if (fg.startswith('n') and len(fg)>1) or fg.startswith('t') or fg.startswith('s'):
        win_entities.append(k)

In [53]:
entities = []
for v, k in ftsim_kvs[:int(len(ftsim_kvs)*.95)]:
    fg = word_flag[k]
    if (fg.startswith('n') and len(fg)>1) or fg.startswith('t') or fg.startswith('s'):
        if k in win_entities:
            entities.append(k)

In [54]:
[e for e in entities]

['宝贵',
 '五年',
 '新春',
 '改革开放',
 '深度',
 '努力实现',
 '以往',
 '船上',
 '筑墙',
 '大海',
 '时',
 '中虎',
 '日内瓦',
 '赵承',
 '智慧',
 '碳达峰',
 '巨变',
 '大江',
 '黑天鹅',
 '中华文明',
 '今天',
 '如今',
 '瑞士',
 '前在',
 '美好世界',
 '中国',
 '潮平海',
 '探索者',
 '共同富裕',
 '习近平',
 '文明',
 '前月日',
 '达沃斯',
 '湖泊',
 '霍小光',
 '太平洋',
 '龙腾虎跃',
 '公正',
 '太平',
 '广度',
 '亚太经合组织',
 '正在',
 '喻',
 '青山',
 '国内外']

### 3.4 确定实体类别

In [55]:
import pandas as pd
e_cate = pd.read_csv("./entity_cate.csv", encoding='utf-8', header=0)
e_cate

,pos,ch_pos,cate
0,nr,人名,人物
1,nr1,汉语姓氏,人物
2,nr2,汉语名字,人物
3,nrj,日语人名,人物
4,nrf,音译人名,人物
5,ns,地名,地址
6,nsf,音译地名,地址
7,nt,机构团体名,机构团体
8,nz,其它专名,其它
9,t,时间词,时间


In [58]:
print("实例\t\t概念")
print("-------------------------")
for e in entities:
    cate = []
    cate = list(e_cate[e_cate['pos'] == word_flag[e]]['cate'])
    if len(cate) == 0:
        cate = ["未知概念"]
    print(e, '\t\t', cate[0])

实例		概念
-------------------------
宝贵 		 人物
五年 		 时间
新春 		 地址
改革开放 		 其它
深度 		 地址
努力实现 		 人物
以往 		 时间
船上 		 处所
筑墙 		 其它
大海 		 地址
时 		 未知概念
中虎 		 其它
日内瓦 		 地址
赵承 		 人物
智慧 		 人物
碳达峰 		 其它
巨变 		 其它
大江 		 地址
黑天鹅 		 其它
中华文明 		 地址
今天 		 时间
如今 		 时间
瑞士 		 地址
前在 		 时间
美好世界 		 其它
中国 		 地址
潮平海 		 地址
探索者 		 人物
共同富裕 		 其它
习近平 		 未知概念
文明 		 人物
前月日 		 时间
达沃斯 		 人物
湖泊 		 地址
霍小光 		 人物
太平洋 		 地址
龙腾虎跃 		 人物
公正 		 人物
太平 		 地址
广度 		 地址
亚太经合组织 		 机构团体
正在 		 时间
喻 		 人物
青山 		 地址
国内外 		 处所
